In [2]:
import geopandas as gpd

import pandas as pd

In [3]:
df = pd.read_excel("/Users/finn/Downloads/amy cook - pusstats data request - 26.02.2024.xlsx")

In [6]:
df.region.unique()

array(['Eastern', 'Midlands & Western', 'Northern', 'Scotland',
       'South Eastern', 'South Western', 'Wales', 'Northern Ireland'],
      dtype=object)

In [16]:
gdf = gpd.read_file("https://raw.githubusercontent.com/EconomicsObservatory/courses/main/datasets/advanced/nuts1.json")

# ['Eastern', 'Midlands & Western', 'Northern', 'Scotland',
#        'South Eastern', 'South Western', 'Wales', 'Northern Ireland']

#  ['SOUTH WEST (ENGLAND)', 'WALES', 'NORTHERN IRELAND',
#        'NORTH EAST (ENGLAND)', 'NORTH WEST (ENGLAND)',
#        'YORKSHIRE AND THE HUMBER', 'EAST MIDLANDS (ENGLAND)',
#        'WEST MIDLANDS (ENGLAND)', 'EAST OF ENGLAND', 'LONDON',
#        'SOUTH EAST (ENGLAND)', 'SCOTLAND']

to_merge = {
    'SOUTH WEST (ENGLAND)': 'South Western',
    'WALES': 'Wales',
    'NORTHERN IRELAND': 'Northern Ireland',
    'NORTH EAST (ENGLAND)': 'Northern',
    'NORTH WEST (ENGLAND)': 'Northern',
    'YORKSHIRE AND THE HUMBER': 'Northern',
    'EAST MIDLANDS (ENGLAND)': 'Midlands & Western',
    'WEST MIDLANDS (ENGLAND)': 'Midlands & Western',
    'EAST OF ENGLAND': 'Eastern',
    'LONDON': 'South Eastern',
    'SOUTH EAST (ENGLAND)': 'South Eastern',
    'SCOTLAND': 'Scotland'
}

gdf['region'] = gdf['NAME_LATN'].map(to_merge)

gdf = gdf.dissolve(by='region')

gdf.to_file("/Users/finn/Documents/GitHub/RADataHub/customRegions.geoJson", driver='GeoJSON')

In [14]:
gdf

,id,LEVL_CODE,NUTS_ID,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,FID,geometry
0,UKK,1,UKK,UK,SOUTH WEST (ENGLAND),SOUTH WEST (ENGLAND),0,0,0,UKK,"MULTIPOLYGON (((-1.66573 51.98749, -1.63700 51..."
1,UKL,1,UKL,UK,WALES,WALES,0,0,0,UKL,"MULTIPOLYGON (((-3.36339 53.35203, -3.16975 53..."
2,UKN,1,UKN,UK,NORTHERN IRELAND,NORTHERN IRELAND,0,0,0,UKN,"MULTIPOLYGON (((-5.97653 55.05660, -5.97080 54..."
3,UKC,1,UKC,UK,NORTH EAST (ENGLAND),NORTH EAST (ENGLAND),0,0,0,UKC,"POLYGON ((-1.34737 54.86069, -1.24223 54.72259..."
4,UKD,1,UKD,UK,NORTH WEST (ENGLAND),NORTH WEST (ENGLAND),0,0,0,UKD,"POLYGON ((-2.68975 55.18906, -2.65030 55.13921..."
5,UKE,1,UKE,UK,YORKSHIRE AND THE HUMBER,YORKSHIRE AND THE HUMBER,0,0,0,UKE,"POLYGON ((-0.79091 54.55948, -0.54857 54.45762..."
6,UKF,1,UKF,UK,EAST MIDLANDS (ENGLAND),EAST MIDLANDS (ENGLAND),0,0,0,UKF,"POLYGON ((0.01738 53.52537, 0.09692 53.49770, ..."
7,UKG,1,UKG,UK,WEST MIDLANDS (ENGLAND),WEST MIDLANDS (ENGLAND),0,0,0,UKG,"POLYGON ((-1.98738 53.21361, -1.86643 53.18290..."
8,UKH,1,UKH,UK,EAST OF ENGLAND,EAST OF ENGLAND,0,0,0,UKH,"POLYGON ((1.67548 52.74269, 1.73639 52.64175, ..."
9,UKI,1,UKI,UK,LONDON,LONDON,0,0,0,UKI,"POLYGON ((-0.24042 51.49011, -0.22283 51.47182..."


In [9]:
gdf.NAME_LATIN

AttributeError: 'GeoDataFrame' object has no attribute 'NAME_LATIN'

In [12]:
gdf.NAME_LATN.unique()

array(['SOUTH WEST (ENGLAND)', 'WALES', 'NORTHERN IRELAND',
       'NORTH EAST (ENGLAND)', 'NORTH WEST (ENGLAND)',
       'YORKSHIRE AND THE HUMBER', 'EAST MIDLANDS (ENGLAND)',
       'WEST MIDLANDS (ENGLAND)', 'EAST OF ENGLAND', 'LONDON',
       'SOUTH EAST (ENGLAND)', 'SCOTLAND'], dtype=object)

In [ ]:
df = pd.read_csv('data/geo_data.geojson')